In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib.patches import Circle, Rectangle
import numba
import numpy as np
import os
import re
from scipy import spatial
from scipy import signal
from scipy import ndimage
from scipy import optimize

%matplotlib qt

In [ ]:
def gen_target(w: float, h: float, dpi: int, loc: str, plot=False):
    '''
    Generate an image of a photogrammetry target.

    Parameters
    ----------
    w
        total image width, m
    h
        total image height, m
    dpi
        output image dpi, converts between matrix coordinates and real width
    loc
        [SW, NW, SE, NE, raft] generates a different pattern for each loc
    '''
    IN_TO_METERS = 0.0254
    w_in = w / IN_TO_METERS
    h_in = h / IN_TO_METERS
    w_px = np.round(w_in * dpi).astype(int)
    h_px = np.round(h_in * dpi).astype(int)

    fig, ax = plt.subplots()
    fig.set_size_inches(w_in, h_in)
    fig.tight_layout()
    fig.subplots_adjust(left=0., right=1., top=1., bottom=0.)

    pattern_size = 8
    pattern = np.random.choice([0.05,0.95], size=(pattern_size,pattern_size))
    pattern[0][0] = 0
    pattern[pattern_size - 1][pattern_size - 1] = 1
    ax.imshow(ndimage.zoom(pattern, h_px / pattern_size, order=0), cmap='Greys')

    # center cross
    c_size = dpi * 0.01 / IN_TO_METERS # 10 cm
    c_thick = dpi * 0.001 / IN_TO_METERS # 1 mm

    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 2), c_size, c_thick, color='silver')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 2, h_px / 2 - c_size / 2), c_thick, c_size, color='silver')) # vert
    ax.add_patch(Rectangle((w_px / 2 - c_size / 2, h_px / 2 - c_thick / 4), c_size, c_thick / 2, color='k')) # horz
    ax.add_patch(Rectangle((w_px / 2 - c_thick / 4, h_px / 2 - c_size / 2), c_thick / 2, c_size, color='k')) # vert

    ax.set_aspect('equal')
    ax.set_xlim([0, w_px])
    ax.set_ylim([0, h_px])
    ax.set_xticks([])
    ax.set_yticks([])
    fig.savefig(f'{loc}_target.png')
    if not plot:
        plt.close()

In [ ]:
# only run this if you need to re-generate photogrammetry targets.
np.random.seed(77777) # generate same noise pattern every time
gen_locs = ['SW', 'NW', 'NE', 'SE', 'raft']
TARGET_W = 0.03 # 30 mm
TARGET_H = TARGET_W
# for loc in gen_locs:
    # gen_target(TARGET_W, TARGET_H, 300, loc, plot=False)

In [ ]:
def calibrate_camera(image_dir, plot=False):
    # https://learnopencv.com/camera-calibration-using-opencv/
    '''
    The general idea is to take a set of test images with a chessboard pattern
    (6x9, i.e. 5x8 intersections) to calculate the distortion parameters of the
    camera/lens combo being used. This will be used to undo distortion effects
    before we can undo projection effects.
    '''
    objpoints = [] # the chessboard vertex locations in the plane of the board
    imgpoints = [] # the chessboard vertex locations in the image space
    
    prev_img_shape = None

    for fname in sorted(glob.glob(os.path.join(image_dir, '**.jpg'), recursive=True)):
        # do camera calibration from chessboard images
        im = image.imread(fname)
        im = np.rot90(im, k=-1) # may not need this, depending on source of images.
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        print(f'Camera calibration corner finding in {fname}:', ret)
        # we will use the img plane points for camera calibration if we successfully found chessboard corners.
        if ret == True:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
            imgpoints.append(corners2)
            im = cv2.drawChessboardCorners(gray, CHESSBOARD, corners2, ret)
        
        if plot:
            plt.figure()
            plt.imshow(im)
            ax = plt.gca()
            ax.set_aspect('equal')

    # Do the actual camera calibration with all the data we gathered.
    h,w = im.shape[:2]
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    optimal_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
        mtx,
        dist, 
        (w,h), 
        1, 
        (w,h)
    )

    return mtx, dist, optimal_camera_matrix

In [ ]:
def xcorr_prep(im):
    '''
    Helper function to apply pre-processing that makes cross-correlation work
    '''
    im_corr = np.sum(im.astype(np.float64), axis=2) # ensure single channel
    im_corr /= im_corr.max() # normalize
    im_corr -= np.mean(im_corr) # detrend
    return im_corr

In [ ]:
def find_template(template_filename, im_warped, avg_px_per_m, ax=None):
    '''
    Returns the pixel coordinates of the best-cross-corr match of the template image in im_warped,
    which ought to be a lens-distortion-free, camera-angle-projection-free image.
    avg_px_per_m is needed to scale the target's known size in m to the image's pixel scale.
    '''
    im_tmp = image.imread(template_filename)
    # number of pixels in a shape with 30mm span:
    target_px = avg_px_per_m * 0.03
    tmp_px = im_tmp.shape[0]
    # zoom the template to the right size in pixels
    tmp = ndimage.zoom(im_tmp, target_px / tmp_px)
    a = xcorr_prep(im_warped)
    b = xcorr_prep(tmp)
    xcorr = signal.fftconvolve(
        a,
        b[::-1, ::-1],
        mode='valid'
    )
    y,x = np.unravel_index(np.argmax(xcorr), xcorr.shape)
    xfound = x + tmp.shape[0] / 2
    yfound = y + tmp.shape[1] / 2

    if ax:
        ax.imshow(a)
        ax.scatter(xfound, yfound, label=template_filename)

    return xfound, yfound

In [ ]:
def find_targets(image_dir, mtx, dist, optimal_camera_matrix, targets=[], plot=False):
    '''
    After we get the camera matrix, let's use it to undo distortion in all test images,
    re-find chessboard corners, then find a homographic transform of the undistorted
    chessboard corners to the ideal, rectified plane of the chessboard (the
    "bird's-eye" view of it).
    Use that and information of how far apart the chessboard intersections really are
    to calculate a px-to-m conversion.
    Finally, do a cross-correlation to find the location of all target images, in m,
    relative to the chessboard 0,0 intersection.
    '''
    files = sorted(glob.glob(os.path.join(image_dir, '**.jpg'), recursive=True))

    image_data = {}
    px_per_ms = []
    for file in files:
        # use camera cal matrix to de-distort all images
        im = image.imread(file)
        im = np.rot90(im, k=-1)
        undistorted_image = cv2.undistort(
            im,
            mtx,
            dist,
            None, 
            optimal_camera_matrix
        )
        # if you get funny results (fake ring images with data from the center around the perimeter),
        # consider this: https://answers.opencv.org/question/28438/undistortion-at-far-edges-of-image/
        # lenses with nonlinear distortion parameters from the center to the edge present problems to
        # the algorithm. apparently it can be mitigated by digital zoom/cropping out data from the
        # corners of the sensor. this effect was very noticeable on my iPhone 8 camera.

        # re-find chessboard corners in undistorted image
        gray = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        print(f'Corner detection after de-distortion in {file}:', ret)
        if ret == True:
            corners2 = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
            undistorted_image_corners = cv2.drawChessboardCorners(undistorted_image, CHESSBOARD, corners2, ret)
        else:
            continue
        # if plot:
        #     plt.figure()
        #     ax = plt.axes()
        #     ax.set_title('After de-distortion')
        #     ax.set_aspect('equal')
        #     ax.grid(True)
        #     ax.imshow(undistorted_image_corners)

        # find the homographic transform that undistorts the found chessboard corners into the rectified, "bird's eye" view of the chessboard.
        this_img = corners2
        # magic number: homographic transform for de-projection sometimes makes resulting image really tiny for some reason.
        # if we didn't scale it back up, we'd have only a few pixels across each chessboard intersection, degrading the
        # quality of the subpixel refinement and eventually the template location xcorr too.
        sf = 3000.
        h, status = cv2.findHomography(
            this_img[:,0,:],
            objp[0,:,:2] * sf + corners2[0][0]
        )
        # deproject chessboard points
        corners_warped = cv2.perspectiveTransform(
            corners2,
            h
        )
        im_warped = cv2.warpPerspective(
            undistorted_image,
            h,
            (undistorted_image.shape[1], undistorted_image.shape[0])
        )
        
        # if plot:
        #     plt.figure()
        #     ax = plt.axes()
        #     ax.set_title('After de-projection')
        #     ax.set_aspect('equal')
        #     ax.grid(True)
        #     ax.imshow(im_warped)

        # locate chessboard corners in deprojected image to calculate pixel scale
        # gray = cv2.cvtColor(im_warped, cv2.COLOR_BGR2GRAY)
        # ret, corners = cv2.findChessboardCorners(
        #     gray,
        #     CHESSBOARD,
        #     cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
        # )
        # print(f'Corner detection after de-projection in {file}:', ret)
        # if ret == True:
        #     corners_warped = cv2.cornerSubPix(gray, corners, (5,5),(-1,-1), criteria)
        #     im_warped_corners = cv2.drawChessboardCorners(im_warped, CHESSBOARD, corners_warped, ret)
        #     im_warped_corners_grey = cv2.cvtColor(im_warped_corners, cv2.COLOR_BGR2GRAY)
        # else:
        #     continue

        # calc pixel scale: get pairs of euclidean distances along each axis.
        # this leaves some information on the table, but is simple and there
        # are enough pairs that the average should be good to submm precision across ~1 m.
        found_corners = corners_warped[:,0,:].reshape((CHESSBOARD[1], CHESSBOARD[0], 2))
        dists0 = np.array([])
        for i in range(found_corners.shape[0]):
            d = np.diff(found_corners[i,:,:], axis=0)
            dists0 = np.concatenate([dists0, np.sqrt((d ** 2).sum(axis=1))])
        dists1 = np.array([])
        for j in range(found_corners.shape[1]):
            d = np.diff(found_corners[:,j,:], axis=0)
            dists1 = np.concatenate([dists1, np.sqrt((d ** 2).sum(axis=1))])
        dists = np.concatenate([dists0, dists1])
        avg_spacing = np.mean(np.abs(dists))
        std_spacing = np.std(np.abs(dists))
        px_per_m = avg_spacing / SQUARE_SIZE
        px_per_ms.append(px_per_m)
        plt.figure()
        # ax = plt.axes()
        # ax.plot(dists)

        # find pixel locs of all targets
        target_dir = os.curdir
        target_locs = {}
        target_locs['chessboard_origin'] = tuple(corners2[0][0].astype(float))
        if plot:
            plt.figure()
            ax = plt.axes()
        else:
            ax = None
        for target in targets:
            x,y = find_template(os.path.join(target_dir, target + '.png'), im_warped, px_per_m, ax=ax)
            target_locs[target] = (x,y)
        if plot:
            ax.legend(loc='best')

        m_per_inch = 0.0254
        for target in [item for item in targets if 'raft_target' not in item]:
            print(f'distance of {target} from raft:', np.linalg.norm(np.array(target_locs[target]) - np.array(target_locs['raft_target'])) / px_per_m / m_per_inch, 'in')

        # save off some data
        image_data[file] = {}
        image_data[file]['px_per_m'] = px_per_m
        image_data[file]['target_locs'] = target_locs

    frac_err = np.std(px_per_ms) / np.mean(px_per_ms)
    avg_px_per_m = np.mean(px_per_ms)
    image_data['source dir'] = os.path.abspath(image_dir)
    image_data['avg_px_per_m'] = avg_px_per_m
    # print('std error induced by pixel cal across span of .7m:', 0.7 * frac_err)

    return image_data

In [ ]:
# global stuff
CHESSBOARD = (5,8)
SQUARE_SIZE = 0.0254 # 25 mm
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# the set of chessboard corner coords in the plane of the chessboard
objp = np.zeros((1, CHESSBOARD[0] * CHESSBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHESSBOARD[0], 0:CHESSBOARD[1]].T.reshape(-1, 2) * SQUARE_SIZE

# calibrate the camera for distortion
mtx, dist, optimal_camera_matrix = calibrate_camera('input', plot=False)

In [ ]:
plt.close('all')
image_data = find_targets('input', mtx, dist, optimal_camera_matrix, targets=['raft_target', 'SW_target', 'SE_target', 'NW_target', 'NE_target'], plot=True)

In [ ]:
print(image_data)